In [2]:
import sys

abspath = os.path.normpath(os.path.dirname(os.path.dirname('..')))

sys.path.append(currentdir)

from toolbox.scrapers import *

url_download_path = os.path.normpath(os.path.join(abspath, 'data', 'urls'))

urls_json = os.path.join(url_download_path, 'hospital_data_urls.json')

hospital_urls = json.load(open(urls_json))




In [3]:

get_duke('duke-university-hospital', hospital_urls)

get_north_carolina_baptist('north-carolina-baptist-hospital', hospital_urls)

get_app('app-regional-health-system', hospital_urls)

get_catawba('catawba-valley-medical-center', hospital_urls)

get_cateret('cateret-health-care', hospital_urls)

get_cone('cone-health', hospital_urls)

get_first('first-health-moore', hospital_urls)

get_first('first-health-montgomery', hospital_urls)

get_iredell('iredell-health', hospital_urls)

get_mission('mission-health', hospital_urls)

get_nhrmc('nhrmc-health', hospital_urls)

get_northern('northern-regional', hospital_urls)

get_novant('novant-health', hospital_urls)

get_wakemed('wakemed-raleigh', hospital_urls)

get_wakemed('wakemed-cary', hospital_urls)

get_vidant('vidant-health', hospital_urls)

get_atrium('atrium-health', hospital_urls)

https://www.dukehealth.org//sites/default/files/general_page/56-2070036_DRaH_standardcharges_drg.csv
56-2070036_DRaH_standardcharges_drg.csv


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\remot\\OneDrive\\Desktop\\Personal\\nc-hospital-transparency\\data\\raw\\duke-university-hospital\\56-2070036_DRaH_standardcharges_drg.csv'

In [ ]:
# ws_access_key_id=os.environ['AWS_ACCESS_KEY_ID']

# ws_access_key_id

aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
    

In [ ]:
aws_secret_access_key

'1bKjkgGBO4YgleMyhhhHr1HgEY6wY33Q/2RA++7T'